In [15]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import linalg as LA
from sklearn import datasets

In [16]:
# load data
df = pd.read_csv('http://rasmuskyng.com/am221_spring18/psets/hw7/banknotes.data',header=None)

In [17]:
# convert labels
df[4] = 2 * df[4] - 1

# get features and labels
x_np = np.array(df.iloc[:,:4])
y_np = np.expand_dims(np.array(df[4]),axis=1)
n,d = x_np.shape

# hyper parameters
lam = 50
t = 1

In [18]:
import torch
from torch.autograd import Variable

In [19]:
x = Variable(torch.DoubleTensor(x_np),requires_grad=False)
y = Variable(torch.DoubleTensor(y_np),requires_grad=False)

In [20]:
def gradient_descent(w_b_xi, lr, t):
    # define variables for gradient descent
    w_np, b_np, xi_np = w_b_xi[:d], w_b_xi[d], w_b_xi[d+1:]
    w = Variable(torch.DoubleTensor(w_np),requires_grad=True)
    b = Variable(torch.DoubleTensor(b_np),requires_grad=True)
    xi = Variable(torch.DoubleTensor(xi_np),requires_grad=True)
    
    # run gradient descent
    for i in range(10000):
        # calculate barrier objective
        barrier_objective = t * (0.5 * w.norm() ** 2 + lam * torch.sum(xi)) \
                      - torch.sum(torch.log(y * x @ w + b + xi - 1)) \
                      - torch.sum(torch.log(xi))
        # run back propagation
        barrier_objective.backward()
        # update the parameters
        w.data -= lr * w.grad.data
        b.data -= lr * b.grad.data
        xi.data -= lr * xi.grad.data

        # manually zero the gradients after running the backward pass
        w.grad.data.zero_()
        b.grad.data.zero_()
        xi.grad.data.zero_()
        
    return np.vstack([w.data.numpy(),np.expand_dims(b.data.numpy(),1),xi.data.numpy()])

In [21]:
def f(X,lam):
    # calculate the true objective
    w, b, xi = X[:d], X[d], X[d+1:]
    return 1/2 * np.linalg.norm(w, ord=2) ** 2 + lam * np.sum(xi)

In [22]:
def acc(X,x,y):
    # calculate the accuracy
    w, b, xi = X[:d], X[d], X[d+1:]
    return np.mean(y * (x @ w + b) >= 1)

In [28]:
# initial feasible solution
w_b_xi = np.expand_dims(np.hstack([0*np.ones(d+1),np.repeat(2,n)]),axis=1)

# more hyper parameters
eps = 0.0001
mu = 1.1
t = 1

# barrier method
accs = []
mus = [1.1, 1.5, 2, 5, 10]
for mu in mus:
    print(mu)
    for i in range(25):
        w_b_xi = gradient_descent(w_b_xi,10**-8/t,t)
        print(i)
        print("t: {} | objective: {} | classification accuracy: {}".format(t,f(w_b_xi,lam),acc(w_b_xi,x_np,y_np)))
        t = mu * t
    accs.append(acc(w_b_xi,x_np,y_np))

1.1
0
t: 1 | objective: 136865.62103624135 | classification accuracy: 0.37244897959183676
1
t: 1.1 | objective: 136529.417169686 | classification accuracy: 0.5342565597667639
2
t: 1.2100000000000002 | objective: 136192.2125845217 | classification accuracy: 0.6399416909620991
3
t: 1.3310000000000004 | objective: 135854.28418061612 | classification accuracy: 0.7004373177842566
4
t: 1.4641000000000006 | objective: 135515.7755352815 | classification accuracy: 0.7419825072886297
5
t: 1.6105100000000008 | objective: 135176.77923031317 | classification accuracy: 0.7638483965014577
6
t: 1.771561000000001 | objective: 134837.36406482928 | classification accuracy: 0.7806122448979592
7
t: 1.9487171000000014 | objective: 134497.58434500627 | classification accuracy: 0.7981049562682215
8
t: 2.1435888100000016 | objective: 134157.48430887505 | classification accuracy: 0.8068513119533528
9
t: 2.357947691000002 | objective: 133817.1008436097 | classification accuracy: 0.8192419825072886
10
t: 2.593742

6
t: 1.434394208202712e+17 | objective: 109142.42228594101 | classification accuracy: 0.8848396501457726
7
t: 7.17197104101356e+17 | objective: 108799.4221697283 | classification accuracy: 0.8848396501457726
8
t: 3.58598552050678e+18 | objective: 108456.42205353882 | classification accuracy: 0.8848396501457726
9
t: 1.79299276025339e+19 | objective: 108113.42193737258 | classification accuracy: 0.8848396501457726
10
t: 8.96496380126695e+19 | objective: 107770.42182122957 | classification accuracy: 0.8848396501457726
11
t: 4.482481900633475e+20 | objective: 107427.42170510975 | classification accuracy: 0.8848396501457726
12
t: 2.2412409503167375e+21 | objective: 107084.42158901317 | classification accuracy: 0.8848396501457726
13
t: 1.1206204751583688e+22 | objective: 106741.42147293978 | classification accuracy: 0.8848396501457726
14
t: 5.6031023757918434e+22 | objective: 106398.42135688958 | classification accuracy: 0.8848396501457726
15
t: 2.8015511878959216e+23 | objective: 106055.421